In [1]:
import pandas as pd
import pymssql
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import numpy as np
from scipy.optimize import curve_fit

In [2]:
def ms_sql_con():
    sql_name = 'voice_ai'
    sql_server = '10.2.4.124'
    sql_login = 'ICECORP\\1c_sql'

    with open('sql.pass','r') as file:
        sql_pass = file.read().replace('\n', '')
        file.close()

    return pymssql.connect(
            server = sql_server,
            user = sql_login,
            password = sql_pass,
            database = sql_name,
        )

In [3]:
def read_sql(query):
    return pd.read_sql(query, con=ms_sql_con(), parse_dates=None)

In [ ]:
def phrases_per_hour(date_from, date_to, phrase):
    query = "select month(record_date) as m,  day(record_date) as d, DATEPART(HOUR, record_date) as h, count(id) as c" 
    query += " from transcribations where "
    query += " record_date > '"+date_from+"' and "
    query += " record_date < '"+date_to+"' and "
    query += " text like '%"+phrase+"%' "
    query += " group by month(record_date), day(record_date), DATEPART(HOUR, record_date)"
    query += " order by month(record_date), day(record_date), DATEPART(HOUR, record_date);"
    df = read_sql(query)
    
    df.drop(['m', 'd'], axis = 1, inplace = True)
    df = df.groupby(df.h).median()
    df.reset_index(level=0, inplace=True)
    
    plt.bar(df.h, df.c)
    plt.show()

In [ ]:
def silence_timing(date_from, date_to):

    def convert_false(val):
        return not val
    
    query = "select"
    query += " sum(end_time - start) as time,"
    query += " DATEPART(HOUR, record_date) as h,"
    query += " linkedid,"
    query += " side,"
    query += " duration"
    query += " from transcribations"
    query += " where"
    query += " record_date > '"+date_from+"' and "
    query += " record_date < '"+date_to+"' and"
    query += " duration > 0 "
    query += " group by record_date, linkedid, side, duration"
    query += " order by linkedid, side;"
    df = read_sql(query)
    df['a'] = df.time*df.side
    df.side = df.side.apply(convert_false)
    df['b'] = df.time*df.side
    df.drop(['time', 'side'], axis = 1, inplace = True)
    df = df.groupby(['h', 'linkedid', 'duration']).sum()
    df.reset_index(level=0, inplace=True)
    df.reset_index(level=0, inplace=True)
    df.reset_index(level=0, inplace=True)
    df.drop(['linkedid'], axis = 1, inplace = True)
    df.reset_index(level=0, inplace=True)
    df['part_a'] = df.a/df.duration
    df['part_b'] = df.b/df.duration
    df['part_silence'] = 1 - df.part_a - df.part_b
    df.drop(['duration', 'a', 'b'], axis = 1, inplace = True)
    df = df.groupby(['h']).mean()
    df.reset_index(level=0, inplace=True)
    df.drop(['index'], axis = 1, inplace = True)
    df.sort_values(['h'], inplace = True)
    df.plot(x = 'h', y = ['part_a', 'part_b', 'part_silence'], style = '.')    

In [ ]:
phrases_per_hour('2021-03-04 00:00:00', '2021-03-05 00:00:00', 'операторы заняты')

In [ ]:
silence_timing('2021-03-01 00:00:00', '2021-03-02 00:00:00')

In [ ]:
### developing

In [4]:
query = "SELECT column_name FROM information_schema.columns WHERE table_name='transcribations';"
df = read_sql(query)
df

,column_name
0,transcribation_date
1,date_y
2,date_m
3,date_d
4,side
5,text
6,start
7,audio_file_name
8,conf
9,end_time


In [13]:
query = "SELECT audio_file_name, duration FROM transcribations where linkedid = '1621227643.435596';"
df = read_sql(query)
df

,audio_file_name,duration
0,a2021-05-17t08:00:45b_c9055316577d_e9091640230...,0.0
1,a2021-05-17t08:00:45b_c9055316577d_e9091640230...,0.0


In [19]:
query = "SELECT * FROM queue WHERE not file_size is null;"
df = read_sql(query)
df

,date,cpu_id,filepath,date_y,date_m,date_d,filename,duration,record_date,source_id,src,dst,linkedid,version,file_size
0,2021-05-18 14:20:25,6,/mnt/share/audio_master/MSK_MRM/REC_IN_OUT/,None,None,None,a2021-05-18t14:07:16b_c9259787512d_e4955226435...,93.94,2021-05-18 14:07:16,2,9259787512,4955226435,1621336033.489833,1,1503084
1,2021-05-18 14:20:26,6,/mnt/share/audio_master/MSK_MRM/REC_IN_OUT/,None,None,None,a2021-05-18t14:07:16b_c9259787512d_e4955226435...,94.08,2021-05-18 14:07:16,2,9259787512,4955226435,1621336033.489833,1,1505324
2,2021-05-18 14:20:26,6,/mnt/share/audio_master/MSK_MRM/REC_IN_OUT/,None,None,None,a2021-05-18t14:08:42b_c9258804878d_e9253089983...,30.44,2021-05-18 14:08:42,2,9258804878,9253089983,1621336120.489884,1,487084
3,2021-05-18 14:20:26,6,/mnt/share/audio_master/MSK_MRM/REC_IN_OUT/,None,None,None,a2021-05-18t14:08:42b_c9258804878d_e9253089983...,30.78,2021-05-18 14:08:42,2,9258804878,9253089983,1621336120.489884,1,492524
4,2021-05-18 14:20:26,9,/mnt/share/audio_master/MSK_MRM/REC_IN_OUT/,None,None,None,a2021-05-18t14:09:26b_c9252904233d_e9859084696...,33.40,2021-05-18 14:09:26,2,9252904233,9859084696,1621336164.489920,1,534444
5,2021-05-18 14:20:26,6,/mnt/share/audio_master/MSK_MRM/REC_IN_OUT/,None,None,None,a2021-05-18t14:09:26b_c9252904233d_e9859084696...,33.52,2021-05-18 14:09:26,2,9252904233,9859084696,1621336164.489920,1,536364


In [10]:
query = "SELECT column_name FROM information_schema.columns WHERE table_name='perf_log';"
df = read_sql(query)
df

,column_name
0,event_date
1,step
2,cpu
3,duration
4,linkedid
5,source_id
6,cores
7,time
8,file_name


In [12]:
query = "select *"
query += " from perf_log "
query += " where linkedid='1621227643.435596';"
df = read_sql(query)
df

,event_date,step,cpu,duration,linkedid,source_id,cores,time,file_name
0,2021-05-17 08:06:48,0,5,0.0,1621227643.435596,2,14,0.023562,
1,2021-05-17 08:06:49,0,4,0.0,1621227643.435596,2,14,0.061769,
